<a href="https://colab.research.google.com/github/lcipolina/escher/blob/master/Img_Class/Img_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip3 install gradio --quiet

     |████████████████████████████████| 1.7 MB 5.1 MB/s 
     |████████████████████████████████| 84 kB 2.7 MB/s 
     |████████████████████████████████| 53 kB 2.5 MB/s 
     |████████████████████████████████| 1.1 MB 52.0 MB/s 
     |████████████████████████████████| 54 kB 3.3 MB/s 
     |████████████████████████████████| 255 kB 59.2 MB/s 
     |████████████████████████████████| 211 kB 69.8 MB/s 
     |████████████████████████████████| 2.0 MB 51.2 MB/s 
     |████████████████████████████████| 271 kB 72.0 MB/s 
     |████████████████████████████████| 144 kB 72.7 MB/s 
     |████████████████████████████████| 94 kB 504 kB/s 
     |████████████████████████████████| 10.9 MB 53.2 MB/s 
     |████████████████████████████████| 58 kB 5.7 MB/s 
     |████████████████████████████████| 79 kB 9.4 MB/s 
     |████████████████████████████████| 43 kB 2.4 MB/s 
     |████████████████████████████████| 61 kB 552 kB/s 
     |████████████████████████████████| 856 kB 61.6 MB/s 
     |████████████████████████

In [5]:
# import libraries
from PIL import Image
from torchvision import models, transforms
import torch
import torch.nn as nn
import torch.nn.functional as F

In [6]:
# Select device
has_cuda = torch.cuda.is_available()
device = torch.device('cpu' if not has_cuda else 'cuda')
print(f"Running on {device}")

Running on cuda


In [7]:
class Discriminator(nn.Module):
    def __init__(self, in_feature_size = 118, batch_size = 1):
        
        super(Discriminator, self).__init__()
        self.batch_size = batch_size
        self.conv1 = nn.Conv2d(1, 64, 3)
        self.bn1 = nn.BatchNorm2d(64, affine=False)
        self.conv2 = nn.Conv2d(64, 128, 3)
        self.bn2 = nn.BatchNorm2d(128, affine=False)
        self.conv3 = nn.Conv2d(128, 256, 3)
        self.bn3 = nn.BatchNorm2d(256, affine=False)
        self.conv4 = nn.Conv2d(256, 512, 3)
        self.bn4 = nn.BatchNorm2d(512, affine=False)
        self.conv5 = nn.Conv2d(512, 1, 3)
        self.fc1 = nn.Linear(in_feature_size * in_feature_size , 64)
        self.fc2 = nn.Linear(64, 8)
        self.fc3 = nn.Linear(8, 2)
    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = self.conv5(x)
        x = x.view(1, -1)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

In [8]:
# Upload image
from google.colab import files
uploaded = files.upload()
# Get the name of uploaded image:
for key in uploaded.keys():
  uploaded_image = key


Saving 7-Photoshop_copy.png to 7-Photoshop_copy.png


In [9]:
#Download weights
!gdown --id 1J0K-_vA42fnGhho7BEt5YQI1_voqzZr5
weights = 'model-v3.ckpt'

Downloading...
From: https://drive.google.com/uc?id=1J0K-_vA42fnGhho7BEt5YQI1_voqzZr5
To: /content/model-v3.ckpt
100% 9.80M/9.80M [00:00<00:00, 25.2MB/s]


In [12]:
# Create model
net = Discriminator()
net = nn.DataParallel(net, device_ids = [0])
net.load_state_dict(torch.load(weights))
net.eval()
labels = ["bad", "good"]
test_transform = transforms.Compose([
            transforms.Resize([128, 128]),
            transforms.RandomHorizontalFlip(),
            transforms.Grayscale(),
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.5), std=(0.5))
        ])
def predict(inp):
  inp = test_transform(inp).unsqueeze(0)
  with torch.no_grad():
    prediction = torch.nn.functional.softmax(net(inp)[0], dim=0)
    confidences = {labels[i]: float(prediction[i]) for i in range(2)}    
  return confidences

In [13]:
import gradio as gr

gr.Interface(fn=predict, 
             inputs=gr.inputs.Image(type="pil"),
             outputs=gr.outputs.Label(num_top_classes=2),).launch(debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://50686.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.


(<fastapi.applications.FastAPI at 0x7f16c35e1ad0>,
 'http://127.0.0.1:7860/',
 'https://50686.gradio.app')